In [ ]:
# HTCondor Parameter Sweep Setup

This notebook helps you:
1. Understand the quantum circuit task
2. Generate parameter combinations for the sweep
3. Verify the setup before submitting to HTCondor

## Task Overview

We'll run a parameter sweep over a simple quantum circuit with two rotation gates (RX and RZ) to explore the parameter space systematically.

In [ ]:
"""
Test the quantum circuit locally before submitting to HTCondor.

This cell runs a single parameter combination to verify:
- The circuit executes correctly
- Dependencies are installed
- Output directory structure works
"""

import pennylane as qml
from pennylane import numpy as np 
import os 

# Test parameters
params = np.array([0.1, 0.02])

# Define quantum device: 1 qubit, 1000 measurement shots
dev = qml.device("default.qubit", wires=1, shots=1000)

@qml.qnode(dev)
def circuit(params):
    """Simple quantum circuit with RX and RZ rotations."""
    qml.RX(params[0], wires=0)
    qml.RZ(params[1], wires=0)
    return qml.sample(qml.PauliZ(0))

# Run circuit
data = circuit(params)
print(f"Circuit executed successfully!")
print(f"Output shape: {data.shape}")
print(f"Sample values: {data[:10]}")

# Test saving (update path to match your setup)
# save_dir = "/data/cvcqml/common/matias/tutorial_pic/cost_reconstruction/{}".format(params)
# os.makedirs(save_dir, exist_ok=True)
# np.save(save_dir, data)
# print(f"Results saved to: {save_dir}")

In [ ]:
## Generate Parameter Grid

Now we'll create a parameter grid for the full sweep. Each combination will be submitted as a separate HTCondor job.

In [ ]:
"""
Generate parameter combinations for HTCondor parameter sweep.

This creates a grid of parameter values that will be submitted
as individual jobs to the HTCondor cluster.
"""

N = 8  # Number of points per dimension (creates N×N = 64 jobs)

# Create parameter ranges
p1 = np.linspace(0., 2*np.pi, N)  # First rotation parameter (RX gate)
p2 = np.linspace(0., 2*np.pi, N)  # Second rotation parameter (RZ gate)

# Write parameter combinations to file
# Update path to match your PIC directory structure
params_file = '/nfs/pic.es/user/m/mbilkis/tutorial-pic/HPC/params.txt'

with open(params_file, 'w') as outf:
    for x in p1:
        for y in p2:
            outf.write("{} {}\n".format(x, y))

print(f"Generated {N*N} parameter combinations")
print(f"Parameters saved to: {params_file}")
print(f"\nFirst few combinations:")
with open(params_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(f"  {line.strip()}")
        else:
            break
print("  ...")